In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define data path
DATA_PATH = "data"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targets = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targets = pickle.load(open(os.path.join(DATA_PATH,'prob_targets.pkl'), 'rb'))
prob_targets_allvisits = pickle.load(open(os.path.join(DATA_PATH,'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targets) == len(seqs)

In [3]:
embedding_matrix = torch.load(os.path.join(DATA_PATH, 'embedding_matrix.pt'))

In [4]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        self.x = seqs
        self.y = targets
    
    def __len__(self):
        return(len(self.x))
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [5]:
dataset = CustomDataset(seqs, prob_targets_allvisits)

In [6]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# total visits excluding last visit per patient, max # diagnosis codes) of type torch.long
        x_masks: a tensor of shape (# total visits excluding last visit per patient, max # diagnosis codes) of type torch.bool
        y: a tensor of shape (# total visits excluding first visit per patient, num higher level categories to predict) of type torch.float
    """
    sequences, targets = zip(*data)

#     y = torch.tensor(targets, dtype=torch.float)
    #import pdb; pdb.set_trace()
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    num_categories = len(targets[0][0])

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    sum_visits = sum(num_visits)
    
    x = torch.zeros((sum_visits - num_patients, max_num_codes), dtype=torch.long)
    y = torch.zeros((sum_visits - num_patients, num_categories), dtype=torch.float)
    x_masks = torch.zeros((sum_visits - num_patients, max_num_codes), dtype=torch.bool)

#     for i_patient, patient in enumerate(sequences):   
#         for j_visit, visit in enumerate(patient):
#             """
#             TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
#             """ 
#             x[i_patient, j_visit] = torch.Tensor(visit)
#             #x_masks[i_patient, j_visit] = torch.Tensor(np.ones(num_codes, dtype=int))
#             x_masks[i_patient, j_visit] = 1
#     import pdb; pdb.set_trace()
    n = 0
    for i,patient in enumerate(sequences):
        for j,visit in enumerate(patient):
            if j == len(patient) - 1:
                break
            for k,code in enumerate(visit):
                x[n,k] = code
                x_masks[n,k] = 1
            n+=1
    n = 0
    for i,patient in enumerate(targets):
        for j,visit in enumerate(patient):
            if j == len(patient) - 1:
                break
            y[n] = torch.tensor(patient[j+1])
            n += 1
    
    
    return x, x_masks, y

In [7]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [8]:
from torch.utils.data.dataset import random_split

train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [9]:
from torch.utils.data import DataLoader

def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    
    '''
    
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        test dataset: test dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train, test and validation dataloaders
    '''
    
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [10]:
def sum_embeddings_with_mask(x, masks):
    """
    Mask select the embeddings for true visits (not padding visits) and then sum the embeddings for each visit up.

    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    
    x = x * masks.unsqueeze(-1)
    x = torch.sum(x, dim = -2)
    return x

In [11]:
def indices_to_multihot(indices, masks, dim):
    #import pdb; pdb.set_trace()
    #indices = indices[masks.any(dim=1)]
    multihot = torch.zeros((indices.shape[0], dim), dtype=torch.float)
    for idx, row in enumerate(indices):
        y_idx = row[masks[idx]].unique()
        multihot[idx] = F.one_hot(y_idx.to(torch.int64), multihot.shape[1]).sum(0)
    return multihot

In [18]:
class EnhancedMLP(nn.Module):
    
    """
    """
    
    def __init__(self, num_codes, num_categories, embedding_matrix):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: number of higher level categories to predict
            embedding_matrix: learned embedding matrix of icd9 descriptions
        """
#         self.padding_idx = 0
        
        #self.embedding = nn.Embedding(num_codes, embedding_dim=128, padding_idx=0)
        self.embedding = nn.Linear(4903, 300)
        self.embedding.weight.data = embedding_matrix
        self.fc = nn.Linear(300, num_categories)
        self.softmax = nn.Softmax(dim=-1)
        
    
    def forward(self, x, masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)
        Outputs:
            logits: logits of shape (batch_size, # diagnosis codes)
        """
#         import pdb; pdb.set_trace()
#         num_codes = self.embedding.weight.shape[0]
#         x = indices_to_multihot(x, masks, num_codes)
#         x[~masks] = self.padding_idx
#         x[masks] += 1
        x = indices_to_multihot(x, masks, 4903)
        x = self.embedding(x)
        x = torch.tanh(x)
#         x = x.sum(dim=1)
        #x = sum_embeddings_with_mask(x, masks)
        logits = self.fc(x)
#         logits = logits.mean(dim=1)
#         probs = self.softmax(logits)
        return logits
    

# load the model here, set embedding_matrix to requires_grad=False (only learn bias vector)
enhanced_mlp = EnhancedMLP(num_codes = len(codes), num_categories=len(sub_categories), embedding_matrix=embedding_matrix)
for param in enhanced_mlp.named_parameters():
    if param[0] == "embedding.weight":
        print(param)
        param[1].requires_grad = False
        print(param)
enhanced_mlp

('embedding.weight', Parameter containing:
tensor([[0.1408, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0120],
        [0.8244, 0.0000, 4.8920,  ..., 0.5002, 0.0000, 4.2985],
        [1.4374, 0.2580, 0.1317,  ..., 2.3848, 2.0505, 0.0000],
        ...,
        [0.8518, 0.0000, 0.3237,  ..., 0.9934, 0.5658, 0.1184],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1696, 2.2176, 0.0000,  ..., 2.6231, 0.6750, 1.3424]],
       requires_grad=True))
('embedding.weight', Parameter containing:
tensor([[0.1408, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0120],
        [0.8244, 0.0000, 4.8920,  ..., 0.5002, 0.0000, 4.2985],
        [1.4374, 0.2580, 0.1317,  ..., 2.3848, 2.0505, 0.0000],
        ...,
        [0.8518, 0.0000, 0.3237,  ..., 0.9934, 0.5658, 0.1184],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1696, 2.2176, 0.0000,  ..., 2.6231, 0.6750, 1.3424]]))


EnhancedMLP(
  (embedding): Linear(in_features=4903, out_features=300, bias=True)
  (fc): Linear(in_features=300, out_features=184, bias=True)
  (softmax): Softmax(dim=-1)
)

In [19]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(baseline_mlp.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(enhanced_mlp.parameters(), weight_decay=0.001)

In [20]:
def eval_model(model, test_loader, k=15, n=-1):
    
    """
    Arguments:
        model: the EnhancedMLP model
        test_loader: validation dataloader
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
    y_pred = torch.LongTensor()
#     y_score = torch.Tensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, masks, y in test_loader:
#             import pdb; pdb.set_trace()
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks)
            y_hat = F.softmax(y_hat, dim=-1)
#             num_labels = y_hat.shape[1]
#             num_categories = torch.count_nonzero(y, dim=1)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
#             predictions = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
#                 for v in y_pred:
#                     if v in y_true:
#                         visit_correct += 1
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
#                 predictions += len(y_true)
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                #print(f'visit {i}: precision: {visit_precision:0.2f} accuracy: {visit_accuracy:0.2f}')
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            #import pdb; pdb.set_trace()
#             precision_k = precision / k
#             accuracy_k = k_correct / predictions
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
#             y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
#             y_hat = (y_hat > 0.5).int()
#             y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
#             y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
#     import pdb; pdb.set_trace()
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [21]:
def train(model, train_loader, test_loader, n_epochs):
    """

    """
    base_cpu, base_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
#         import pdb; pdb.set_trace()
        for x, masks, y in train_loader:

            y_hat = model(x, masks)
#             mask_idxs = masks.sum(dim=1) - 1
#             y_hat = y_hat[range(len(masks)), mask_idxs]
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print_cpu_usage()
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, test_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()

In [22]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [23]:
n_epochs = 100
%time train(enhanced_mlp, train_loader, val_loader, n_epochs)

CPU: 20.61
RAM %: 65.0
CPU: 20.68
RAM %: 65.0
Epoch: 1 	 Training Loss: 4.020759
Epoch: 1 	 Validation precision@k5: 0.4250, accuracy@k5: 0.2092
Epoch: 1 	 Validation precision@k10: 0.4575, accuracy@k10: 0.3798
Epoch: 1 	 Validation precision@k15: 0.5303, accuracy@k15: 0.5113
Epoch: 1 	 Validation precision@k20: 0.5996, accuracy@k20: 0.5969
Epoch: 1 	 Validation precision@k25: 0.6772, accuracy@k25: 0.6770
Epoch: 1 	 Validation precision@k30: 0.7372, accuracy@k30: 0.7372
CPU: 20.91
RAM %: 65.0
Epoch: 2 	 Training Loss: 3.972974
Epoch: 2 	 Validation precision@k5: 0.4807, accuracy@k5: 0.2372
Epoch: 2 	 Validation precision@k10: 0.4761, accuracy@k10: 0.3974
Epoch: 2 	 Validation precision@k15: 0.5397, accuracy@k15: 0.5213
Epoch: 2 	 Validation precision@k20: 0.6237, accuracy@k20: 0.6209
Epoch: 2 	 Validation precision@k25: 0.6743, accuracy@k25: 0.6742
Epoch: 2 	 Validation precision@k30: 0.7389, accuracy@k30: 0.7389
CPU: 21.00
RAM %: 64.8
Epoch: 3 	 Training Loss: 3.966880
Epoch: 3 	 Vali

CPU: 21.85
RAM %: 65.0
Epoch: 19 	 Training Loss: 3.886271
Epoch: 19 	 Validation precision@k5: 0.5118, accuracy@k5: 0.2583
Epoch: 19 	 Validation precision@k10: 0.5080, accuracy@k10: 0.4262
Epoch: 19 	 Validation precision@k15: 0.5591, accuracy@k15: 0.5403
Epoch: 19 	 Validation precision@k20: 0.6296, accuracy@k20: 0.6269
Epoch: 19 	 Validation precision@k25: 0.6955, accuracy@k25: 0.6953
Epoch: 19 	 Validation precision@k30: 0.7565, accuracy@k30: 0.7565
CPU: 21.85
RAM %: 65.0
Epoch: 20 	 Training Loss: 3.882404
Epoch: 20 	 Validation precision@k5: 0.4461, accuracy@k5: 0.2275
Epoch: 20 	 Validation precision@k10: 0.4925, accuracy@k10: 0.4129
Epoch: 20 	 Validation precision@k15: 0.5541, accuracy@k15: 0.5356
Epoch: 20 	 Validation precision@k20: 0.6268, accuracy@k20: 0.6242
Epoch: 20 	 Validation precision@k25: 0.6882, accuracy@k25: 0.6881
Epoch: 20 	 Validation precision@k30: 0.7418, accuracy@k30: 0.7418
CPU: 21.84
RAM %: 64.9
Epoch: 21 	 Training Loss: 3.884828
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.7605, accuracy@k30: 0.7605
CPU: 22.47
RAM %: 64.8
Epoch: 37 	 Training Loss: 3.862666
Epoch: 37 	 Validation precision@k5: 0.5114, accuracy@k5: 0.2554
Epoch: 37 	 Validation precision@k10: 0.4947, accuracy@k10: 0.4143
Epoch: 37 	 Validation precision@k15: 0.5625, accuracy@k15: 0.5433
Epoch: 37 	 Validation precision@k20: 0.6366, accuracy@k20: 0.6339
Epoch: 37 	 Validation precision@k25: 0.7140, accuracy@k25: 0.7139
Epoch: 37 	 Validation precision@k30: 0.7634, accuracy@k30: 0.7634
CPU: 22.52
RAM %: 64.9
Epoch: 38 	 Training Loss: 3.864873
Epoch: 38 	 Validation precision@k5: 0.5138, accuracy@k5: 0.2604
Epoch: 38 	 Validation precision@k10: 0.5026, accuracy@k10: 0.4222
Epoch: 38 	 Validation precision@k15: 0.5586, accuracy@k15: 0.5399
Epoch: 38 	 Validation precision@k20: 0.6383, accuracy@k20: 0.6356
Epoch: 38 	 Validation precision@k25: 0.7111, accuracy@k25: 0.7110
Epoch: 38 	 Validation precision@k30: 0.7657, accuracy@k30: 0.7657
CPU: 22.47
RAM 

Epoch: 54 	 Validation precision@k25: 0.7084, accuracy@k25: 0.7082
Epoch: 54 	 Validation precision@k30: 0.7635, accuracy@k30: 0.7635
CPU: 23.29
RAM %: 64.4
Epoch: 55 	 Training Loss: 3.852606
Epoch: 55 	 Validation precision@k5: 0.5128, accuracy@k5: 0.2583
Epoch: 55 	 Validation precision@k10: 0.5064, accuracy@k10: 0.4245
Epoch: 55 	 Validation precision@k15: 0.5681, accuracy@k15: 0.5487
Epoch: 55 	 Validation precision@k20: 0.6423, accuracy@k20: 0.6395
Epoch: 55 	 Validation precision@k25: 0.7116, accuracy@k25: 0.7115
Epoch: 55 	 Validation precision@k30: 0.7677, accuracy@k30: 0.7677
CPU: 23.23
RAM %: 64.4
Epoch: 56 	 Training Loss: 3.854883
Epoch: 56 	 Validation precision@k5: 0.5191, accuracy@k5: 0.2621
Epoch: 56 	 Validation precision@k10: 0.4958, accuracy@k10: 0.4183
Epoch: 56 	 Validation precision@k15: 0.5522, accuracy@k15: 0.5343
Epoch: 56 	 Validation precision@k20: 0.6299, accuracy@k20: 0.6273
Epoch: 56 	 Validation precision@k25: 0.7034, accuracy@k25: 0.7033
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.6437, accuracy@k20: 0.6410
Epoch: 72 	 Validation precision@k25: 0.7145, accuracy@k25: 0.7144
Epoch: 72 	 Validation precision@k30: 0.7600, accuracy@k30: 0.7600
CPU: 22.46
RAM %: 64.3
Epoch: 73 	 Training Loss: 3.851617
Epoch: 73 	 Validation precision@k5: 0.4777, accuracy@k5: 0.2450
Epoch: 73 	 Validation precision@k10: 0.5014, accuracy@k10: 0.4227
Epoch: 73 	 Validation precision@k15: 0.5665, accuracy@k15: 0.5479
Epoch: 73 	 Validation precision@k20: 0.6399, accuracy@k20: 0.6372
Epoch: 73 	 Validation precision@k25: 0.7096, accuracy@k25: 0.7095
Epoch: 73 	 Validation precision@k30: 0.7668, accuracy@k30: 0.7668
CPU: 22.38
RAM %: 64.6
Epoch: 74 	 Training Loss: 3.848106
Epoch: 74 	 Validation precision@k5: 0.5062, accuracy@k5: 0.2600
Epoch: 74 	 Validation precision@k10: 0.4933, accuracy@k10: 0.4162
Epoch: 74 	 Validation precision@k15: 0.5489, accuracy@k15: 0.5309
Epoch: 74 	 Validation precision@k20: 0.6263, accuracy@k20: 0.6236
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.5475, accuracy@k15: 0.5294
Epoch: 90 	 Validation precision@k20: 0.6342, accuracy@k20: 0.6316
Epoch: 90 	 Validation precision@k25: 0.7027, accuracy@k25: 0.7026
Epoch: 90 	 Validation precision@k30: 0.7594, accuracy@k30: 0.7594
CPU: 22.54
RAM %: 64.7
Epoch: 91 	 Training Loss: 3.844197
Epoch: 91 	 Validation precision@k5: 0.4952, accuracy@k5: 0.2523
Epoch: 91 	 Validation precision@k10: 0.4952, accuracy@k10: 0.4171
Epoch: 91 	 Validation precision@k15: 0.5639, accuracy@k15: 0.5448
Epoch: 91 	 Validation precision@k20: 0.6457, accuracy@k20: 0.6430
Epoch: 91 	 Validation precision@k25: 0.7107, accuracy@k25: 0.7106
Epoch: 91 	 Validation precision@k30: 0.7607, accuracy@k30: 0.7607
CPU: 22.66
RAM %: 64.6
Epoch: 92 	 Training Loss: 3.841235
Epoch: 92 	 Validation precision@k5: 0.5221, accuracy@k5: 0.2626
Epoch: 92 	 Validation precision@k10: 0.5051, accuracy@k10: 0.4266
Epoch: 92 	 Validation precision@k15: 0.5581, accuracy@k15: 0.5400
Epoch: 92 	 Val

In [24]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(enhanced_mlp, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.4926, accuracy@k5: 0.2505
Validation precision@k10: 0.4911, accuracy@k10: 0.4096
Validation precision@k15: 0.5512, accuracy@k15: 0.5302
Validation precision@k20: 0.6261, accuracy@k20: 0.6222
Validation precision@k25: 0.7001, accuracy@k25: 0.6999
Validation precision@k30: 0.7492, accuracy@k30: 0.7492
